### **Aviso**: Puede descargar este archivo y correrlo en Google Colab.

# Librerías necesarias

Para que el paquete funcione de manera correcta es necesario que antes de implementarlo importe dos librerías bastante conocidas para cualquier programador en python: `numpy`, `random` y `prettytable`. Algunas de sus clases y funciones se utilizan para realizar la busqueda tabú.

In [22]:
#Se necesita importar la librería numpy y random
import numpy as np
import random
from prettytable import PrettyTable

# Codigo de la clase pricipal

El paquete *`knapsack_tabu`* se compone de una clase llamada `ProblemaMochilaMod`. Se puede crear una instancia u objeto de esta clase con los siguientes argumentos;

*   ítems: Es una lista con los nombres de los objetos candidatos
*   utilidades: Es un diccionario con los nombres de los objetos candidatos como llaves y las utilidades de los objetos como valores
*   pesos: Es un diccionario con los nombres de los objetos candidatos como llaves y los pesos de los objetos como valores
*   límite: Es el peso límite que soporta la mochila

Puede revisar los metodos de la clase acudiendo directamente a la documentación del mismo.

**Cometarios importantes**

1.   Para solucionar el problema representamos a cada solución o combinación de objetpos como un vector binario. Donde cada entrada representa un objeto candidato y su entrada si está en la combinación (=1) o si no lo está (=0).
2.   Dada la representación anterior, podemos asociar un indice numerico a cada combinación de objetos mediante. Solo basta considerar el vector binario como un numero binario.



In [16]:
class ProblemaMochilaMod:
    def __init__(self, ítems, utilidades, pesos, límite):
        self.ítems = ítems
        self.utilidades = np.array([list(utilidades.values())[i] for i in range(len(utilidades))])
        self.pesos = np.array([list(pesos.values())[i] for i in range(len(pesos))])
        self.límite = límite
        self.num_items = len(self.ítems)

    def vector_ítems(self, vector):
        """
        Genera una lista de ítems seleccionados basándose en un vector binario.

        Parámetros:
        - vector (list): Un vector binario donde cada elemento indica la decisión de incluir (1) o no incluir (0) un ítem.

        Retorno:
        - list: Una lista de ítems asociados a las posiciones donde el valor en el vector es igual a 1.
        """
        return [self.ítems[i] for i in range(len(vector)) if vector[i] == 1]


    def generar_solucion_aleatoria(self):
        """
        Genera una solución aleatoria para el problema de la mochila.

        Retorna:
        - np.array: Un array NumPy que representa una solución aleatoria, donde cada elemento indica la decisión de incluir (1) o no incluir (0) un ítem en la mochila.
        """
        return np.array([random.randint(0, 1) for _ in range(self.num_items)])


    def mejor_cambio(self, indices, v_actual):
        """
        Determina el mejor índice de cambio en función de la relación utilidad/peso.

        Parámetros:
        - indices (list): Lista de índices disponibles para el cambio.
        - v_actual (list): Vector binario actual que representa la solución actual.

        Retorna:
        - int: El índice del mejor cambio en la lista de índices.
        """
        # Inicializar el índice y la utilidad del mejor cambio con el primer índice del conjunto.
        i_best = indices[0]
        u_best = (self.utilidades[i_best] / self.pesos[i_best]) * ((-1) ** (v_actual[i_best]))

        # Iterar sobre los índices restantes y actualizar el índice del mejor cambio si se encuentra una utilidad mejor.
        for i in indices:
            # Calcular la utilidad para el índice actual.
            u_i = (self.utilidades[i] / self.pesos[i]) * ((-1) ** (v_actual[i]))

            # Verificar si la utilidad actual es mayor o igual a la utilidad del mejor cambio.
            if u_i >= u_best:
                i_best = i

        # Retornar el índice del mejor cambio.
        return i_best


    def utilidad(self, vector):
        """
        Calcula la utilidad total asociada a un vector binario dado.

        Parámetros:
        - vector (list): Un vector binario donde cada elemento indica la decisión de incluir (1) o no incluir (0) un ítem.

        Retorna:
        - float: La utilidad total calculada multiplicando el vector binario por el vector de utilidades y sumando los productos resultantes.
        """
        return self.utilidades.dot(vector)


    def num_solucion(self, vector):
        """
        Convierte un vector binario en un número entero.

        Parámetros:
        - vector (list): Un vector binario donde cada elemento indica la decisión de incluir (1) o no incluir (0) un ítem.

        Retorna:
        - int: El número entero resultante de la conversión del vector binario.
        """
        # Inicializar una cadena vacía para almacenar la representación binaria del número.
        n_bin = ""

        # Iterar sobre cada elemento del vector binario.
        for bin_value in vector:
            # Concatenar la representación en cadena de cada elemento al final de n_bin.
            n_bin = n_bin + str(bin_value)

        # Convertir la cadena binaria a un número entero en base 2.
        return int(n_bin, 2)


    def tabu_search(self, max_iter=200, L=8):
        """
        Implementa el algoritmo de búsqueda tabú para resolver el problema de la mochila.

        Parámetros:
        - max_iter (int): Número máximo de iteraciones del algoritmo (default: 20).
        - L (int): Longitud máxima de la lista tabú (default: 8).

        Retorna:
        - tuple: Una tupla que contiene la mejor solución encontrada, el número asociado a la solución, el peso total y la utilidad total.
        """

        # Inicialización de variables
        tabu_list = []  # Lista tabú para almacenar índices prohibidos temporalmente
        c = 1  # Contador de iteraciones
        X = self.generar_solucion_aleatoria()  # Generar solución aleatoria inicial
        X_best = X.copy()  # Mejor solución actual
        Curl_W = X_best.dot(self.pesos)  # Peso total de la mejor solución actual

        # Bucle principal de la búsqueda tabú
        while c <= max_iter:
            N = {i for i in range(self.num_items)}  # Conjunto de todos los índices disponibles

            # Excluir índices en la lista tabú del conjunto de índices disponibles
            if len(tabu_list) != 0:
                for i in tabu_list:
                    N.remove(i)

            aux = set()  # Conjunto auxiliar para almacenar índices que deben ser excluidos
            for i in N:
                # Verificar si se puede incluir el ítem i sin exceder el límite de peso
                if (X[i] == 0) and (Curl_W + self.pesos[i] > self.límite):
                    aux.add(i)

            N = N - aux  # Actualizar conjunto de índices disponibles

            if len(N) != 0:
                # Determinar el mejor índice de cambio utilizando la función mejor_cambio
                i_posible = self.mejor_cambio(list(N), X)

                # Actualizar la lista tabú
                if len(tabu_list) < L:
                    tabu_list.append(i_posible)
                else:
                    tabu_list.pop(0)
                    tabu_list.append(i_posible)

                # Realizar el cambio en la solución actual
                X[i_posible] = 1 - X[i_posible]

                # Actualizar el peso total de la solución actual
                if X[i_posible] == 1:
                    Curl_W = Curl_W + self.pesos[i_posible]
                else:
                    Curl_W = Curl_W - self.pesos[i_posible]

                # Actualizar la mejor solución si la nueva solución es mejor
                if self.utilidad(X) > self.utilidad(X_best):
                    X_best = X.copy()
            c += 1
        # Retornar la mejor solución encontrada junto con información adicional
        return X_best, self.utilidad(X_best), self.num_solucion(X_best), Curl_W

    def ejecutar_con_varios_L(self, max_iter=20, max_L=8):
        resultados = PrettyTable(['L', 'Mejor Solución Binaria', 'Mejor Solución Traducida', 'Utilidad'])
        for L in range(1, max_L + 1):
            mejor_solucion_binaria, utilidad = self.tabu_search(max_iter=max_iter, L=L)[:2]

            # Obtener nombres de ítems para la mejor solución binaria
            nombres_mejor_solucion = self.vector_ítems(mejor_solucion_binaria.tolist())

            # Obtener utilidades de la mejor solución binaria
            utilidades_mejor_solucion = self.utilidades[mejor_solucion_binaria.astype(bool)].tolist()

            resultados.add_row([L, mejor_solucion_binaria.tolist(), nombres_mejor_solucion, utilidad])

            # Ajustes manuales de ancho de columnas
            resultados._max_width = {"Mejor Solución Binaria": 20, "Mejor Solución Traducida": 30, "Utilidades": 20, "Utilidad Total": 15}
        return resultados


# Ejemplos particulares



In [17]:
# Ejemplo 1

# Cree los objetos para cada uno de los argumentos de la clase ProblemaMochilaMod (puede revisarlos más arriba)
item = ["linterna", "libro", "baterías", "lata", "bolsa de dormir", "mapa", "celular", "encendedor", "asador", "computadora"]
utilidad = {"linterna":10, "libro":2, "baterías":4, "lata":7, "bolsa de dormir": 20, "mapa":6, "celular":7, "encendedor": 8, "asador":6, "computadora":7}
peso = {"linterna":3, "libro":5, "baterías":1, "lata":3, "bolsa de dormir": 8, "mapa":1, "celular": 2, "encendedor":1, "asador":10, "computadora":1}
lím = 10
# Cree una instancia de la clase mochila con los objetos como argumentos. Llame a la clase de manera tal que identifique al caso del problema de la mochila
# que quiere resolver
Caso_1=ProblemaMochilaMod(ítems=item, utilidades=utilidad, pesos=peso, límite=lím)

In [18]:
# Ejemplo 2
item_2 = ["linterna", "libro", "baterías", "lata", "bolsa de dormir"]
utilidad_2 = {"linterna":10, "libro":2, "baterías":4, "lata":7, "bolsa de dormir": 20}
peso_2 = {"linterna":3, "libro":5, "baterías":2, "lata":4, "bolsa de dormir": 8}
lím_2 = 9

Caso_2 = ProblemaMochilaMod(ítems=item_2, utilidades=utilidad_2, pesos=peso_2, límite=lím_2)

In [19]:
#Ejemplo 3
item_3  = ["Caja azul","Caja amarilla","Caja naranja","Caja gris", "Caja verde"]
precio= {"Caja azul":2, "Caja amarilla":10, "Caja naranja":1, "Caja gris":2, "Caja verde":4 }
peso_3= {"Caja azul":2, "Caja amarilla":4, "Caja naranja":1, "Caja gris":1, "Caja verde":12 }
lim_3= 15
Caso_3 = ProblemaMochilaMod(ítems=item_3, utilidades=precio, pesos=peso_3, límite=lim_3)


# Soluciones de los ejemplos

In [20]:
#Solución ejemplo 1

# Para hallar una solución o obtener una aproximación al problema de la mochila asociado al objeto que creo
# use el metodo tabu_search para optenerla. Recuerde que debe proporcinar un limite de iteraciones y un tiempo de vida
# que se traduce como el tamaño de la lista tabu.
mejor_solucion_encontrada = Caso_1.tabu_search(30,8)
# El resultado es una tuplaque contiene: vector solución, la utilidad de la solución, el indice de la solución y el peso total de la solucion.
# Además puede usar el metodo vector_ítems para obtener la lista de objetos candidatos que conforman la solución.
mejor_solucion_encontrada, Caso_1.vector_ítems(mejor_solucion_encontrada[0])

((array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0]), 64, 1020, 9),
 ['linterna',
  'libro',
  'baterías',
  'lata',
  'bolsa de dormir',
  'mapa',
  'celular',
  'encendedor'])

In [27]:
#Metodo para revisar soluciones con varios tiempos de vida (tamaño de la lista tabú)
Caso_1.ejecutar_con_varios_L(max_iter=200, max_L=8)

L,Mejor Solución Binaria,Mejor Solución Traducida,Utilidad
1,"[0, 1, 0, 0, 1, 0, 0, 1, 1, 1]","['libro', 'bolsa de dormir', 'encendedor', 'asador', 'computadora']",43
2,"[1, 0, 1, 0, 0, 1, 1, 1, 0, 1]","['linterna', 'baterías', 'mapa', 'celular', 'encendedor', 'computadora']",42
3,"[1, 0, 1, 1, 0, 1, 0, 1, 0, 1]","['linterna', 'baterías', 'lata', 'mapa', 'encendedor', 'computadora']",42
4,"[1, 0, 1, 0, 0, 1, 1, 1, 0, 1]","['linterna', 'baterías', 'mapa', 'celular', 'encendedor', 'computadora']",42
5,"[1, 0, 1, 0, 0, 1, 1, 1, 0, 1]","['linterna', 'baterías', 'mapa', 'celular', 'encendedor', 'computadora']",42
6,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1]","['linterna', 'libro', 'baterías', 'lata', 'bolsa de dormir', 'mapa', 'celular', 'encendedor', 'computadora']",71
7,"[1, 1, 0, 0, 1, 1, 1, 1, 0, 0]","['linterna', 'libro', 'bolsa de dormir', 'mapa', 'celular', 'encendedor']",53
8,"[1, 1, 1, 0, 0, 1, 1, 1, 1, 1]","['linterna', 'libro', 'baterías', 'mapa', 'celular', 'encendedor', 'asador', 'computadora']",50


In [9]:
#Solución ejemplo 2
mejor_solucion_encontrada_2 = Caso_2.tabu_search(15,8)
mejor_solucion_encontrada_2, Caso_2.vector_ítems(mejor_solucion_encontrada_2[0])

((array([1, 1, 1, 1, 0]), 23, 30, 8),
 ['linterna', 'libro', 'baterías', 'lata'])

In [8]:
#Solución ejemplo 3
mejor_solucion_encontrada_3 = Caso_3.tabu_search(30,8)
mejor_solucion_encontrada_3, Caso_3.vector_ítems(mejor_solucion_encontrada_3[0])

((array([1, 1, 1, 1, 0]), 15, 30, 6),
 ['Caja azul', 'Caja amarilla', 'Caja naranja', 'Caja gris'])